## Setup and Configuration

In [ ]:
# Import required libraries
import sys
import os
from pathlib import Path

# Setup path for utils module - works from any directory
def find_utils_path():
    """Find utils path regardless of current working directory"""
    possible_paths = [
        Path(__file__).parent.parent / 'utils' if '__file__' in dir() else None,
        Path.cwd() / 'notebooks' / 'utils',
        Path.cwd().parent / 'utils',
        Path('/workspace/repo/notebooks/utils'),
        Path('/opt/app-root/src/notebooks/utils'),
        Path('/opt/app-root/src/openshift-aiops-platform/notebooks/utils'),
    ]
    for p in possible_paths:
        if p and p.exists() and (p / 'common_functions.py').exists():
            return str(p)
    current = Path.cwd()
    for _ in range(5):
        utils_path = current / 'notebooks' / 'utils'
        if utils_path.exists():
            return str(utils_path)
        current = current.parent
    return None

utils_path = find_utils_path()
if utils_path:
    sys.path.insert(0, utils_path)
    print(f"✅ Utils path found: {utils_path}")
else:
    print("⚠️ Utils path not found - will use fallback implementations")

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime, timedelta
import joblib
import warnings
warnings.filterwarnings('ignore')

# Machine learning libraries
from sklearn.ensemble import IsolationForest
from sklearn.preprocessing import StandardScaler, RobustScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline  # ✨ Added for KServe compatibility

# Try to import common functions, with fallback
try:
    from common_functions import (
        setup_environment, print_environment_info,
        generate_synthetic_timeseries, validate_data_quality,
        plot_metric_overview, save_processed_data, load_processed_data
    )
    print("✅ Common functions imported")
except ImportError as e:
    print(f"⚠️ Common functions not available: {e}")
    print("   Using minimal fallback implementations")
    
    def setup_environment():
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        os.makedirs('/opt/app-root/src/models/anomaly-detection', exist_ok=True)
        return {'data_dir': '/opt/app-root/src/data', 'models_dir': '/opt/app-root/src/models'}
    
    def print_environment_info(env_info):
        print(f"📁 Data dir: {env_info.get('data_dir', 'N/A')}")
    
    def generate_synthetic_timeseries(metric_name, duration_hours=24, interval_minutes=1, 
                                      add_anomalies=True, anomaly_probability=0.02):
        num_points = int(duration_hours * 60 / interval_minutes)
        timestamps = pd.date_range(end=datetime.now(), periods=num_points, freq=f'{interval_minutes}min')
        values = np.random.normal(50, 10, num_points)
        if add_anomalies:
            anomaly_idx = np.random.choice(num_points, int(num_points * anomaly_probability), replace=False)
            values[anomaly_idx] *= np.random.choice([0.3, 3.0], len(anomaly_idx))
        df = pd.DataFrame({'timestamp': timestamps, 'value': values, 'metric': metric_name, 'is_anomaly': False})
        if add_anomalies:
            df.loc[anomaly_idx, 'is_anomaly'] = True
        return df
    
    def save_processed_data(data, filename):
        os.makedirs('/opt/app-root/src/data/processed', exist_ok=True)
        filepath = f'/opt/app-root/src/data/processed/{filename}'
        if hasattr(data, 'to_parquet'):
            data.to_parquet(filepath)
        print(f"💾 Saved: {filepath}")

print("✅ Libraries imported successfully")
print(f"🔬 Scikit-learn available with Pipeline support")
print(f"📊 Pandas version: {pd.__version__}")

In [ ]:
# Set up environment
env_info = setup_environment()
print_environment_info(env_info)

# Configuration for Isolation Forest
ISOLATION_FOREST_CONFIG = {
    'contamination': 0.05,  # Expected proportion of anomalies
    'n_estimators': 200,    # Number of trees
    'max_samples': 'auto',  # Number of samples to draw
    'max_features': 1.0,    # Number of features to draw
    'random_state': 42      # For reproducibility
}

TARGET_METRICS = [
    # Resource Metrics (5) - CPU and Memory
    'node_memory_utilization',      # ✅ 91 points - Node memory %
    'pod_cpu_usage',                # ✅ 5,076 points - Pod CPU cores
    'pod_memory_usage',             # ✅ 5,071 points - Pod memory bytes
    'alt_cpu_usage',                # ✅ 5,077 points - Alternative CPU metric
    'alt_memory_usage',             # ✅ 5,071 points - Alternative memory metric
    
    # Stability Metrics (3) - Restarts and Availability
    'container_restart_count',      # ✅ 10,668 points - Total restarts
    'container_restart_rate_1h',    # ✅ 7,027 points - Restarts per hour
    'deployment_unavailable',       # ✅ 2,392 points - Unavailable replicas
    
    # Pod Status Metrics (4) - Pod lifecycle
    'namespace_pod_count',          # ✅ 962 points - Pods per namespace
    'pods_pending',                 # ✅ 962 points - Pending pods
    'pods_running',                 # ✅ 962 points - Running pods
    'pods_failed',                  # ✅ 962 points - Failed pods
    
    # Storage Metrics (2) - PVC and quotas
    'persistent_volume_usage',      # ✅ 130 points - PVC usage %
    'cluster_resource_quota',       # ✅ 130 points - Resource quotas
    
    # Control Plane Metrics (2) - API server health
    'apiserver_request_total',      # ✅ 13 points - API request rate
    'apiserver_error_rate',         # ✅ 13 points - API error rate %
]

# PromQL queries for each metric (for reference)
PROMETHEUS_QUERIES = {
    'node_memory_utilization': 'instance:node_memory_utilisation:ratio * 100',
    'pod_cpu_usage': 'sum by (pod, namespace) (node_namespace_pod_container:container_cpu_usage_seconds_total:sum_irate)',
    'pod_memory_usage': 'sum by (pod, namespace) (container_memory_working_set_bytes{container!="POD", container!=""})',
    'alt_cpu_usage': 'sum(rate(container_cpu_usage_seconds_total{container!="POD", container!=""}[5m])) by (pod, namespace)',
    'alt_memory_usage': 'sum(container_memory_rss{container!="POD", container!=""}) by (pod, namespace)',
    'container_restart_count': 'sum by (pod, namespace, container) (kube_pod_container_status_restarts_total)',
    'container_restart_rate_1h': 'sum by (pod, namespace) (increase(kube_pod_container_status_restarts_total[1h]))',
    'deployment_unavailable': 'sum by (deployment, namespace) (kube_deployment_status_replicas_unavailable)',
    'namespace_pod_count': 'sum by (namespace) (kube_pod_status_phase)',
    'pods_pending': 'sum by (namespace) (kube_pod_status_phase{phase="Pending"})',
    'pods_running': 'sum by (namespace) (kube_pod_status_phase{phase="Running"})',
    'pods_failed': 'sum by (namespace) (kube_pod_status_phase{phase="Failed"})',
    'persistent_volume_usage': 'kubelet_volume_stats_used_bytes / kubelet_volume_stats_capacity_bytes * 100',
    'cluster_resource_quota': 'kube_resourcequota',
    'apiserver_request_total': 'sum(rate(apiserver_request_total[5m]))',
    'apiserver_error_rate': 'sum(rate(apiserver_request_total{code=~"5.."}[5m])) / sum(rate(apiserver_request_total[5m])) * 100',
}

# =============================================================================
# DISPLAY CONFIGURATION SUMMARY
# =============================================================================

print("=" * 70)
print("🎯 ISOLATION FOREST CONFIGURATION")
print("=" * 70)

print(f"\n🌲 Model Configuration:")
print(f"   Trees: {ISOLATION_FOREST_CONFIG['n_estimators']}")
print(f"   Contamination: {ISOLATION_FOREST_CONFIG['contamination']} ({ISOLATION_FOREST_CONFIG['contamination']*100:.0f}% expected anomalies)")
print(f"   Max samples: {ISOLATION_FOREST_CONFIG['max_samples']}")

print(f"\n📊 Target Metrics: {len(TARGET_METRICS)} (ALL VERIFIED WORKING)")

# Group metrics by category
categories = {
    'Resource (CPU/Memory)': [m for m in TARGET_METRICS if any(x in m for x in ['cpu', 'memory'])],
    'Stability (Restarts)': [m for m in TARGET_METRICS if any(x in m for x in ['restart', 'unavailable'])],
    'Pod Status': [m for m in TARGET_METRICS if any(x in m for x in ['pod', 'namespace']) and 'cpu' not in m and 'memory' not in m],
    'Storage': [m for m in TARGET_METRICS if any(x in m for x in ['volume', 'quota', 'persistent'])],
    'Control Plane': [m for m in TARGET_METRICS if 'apiserver' in m],
}

for category, metrics in categories.items():
    if metrics:
        print(f"\n   📁 {category}: {len(metrics)} metrics")
        for m in metrics:
            print(f"      ✅ {m}")

print("\n" + "=" * 70)
print("✅ Configuration complete - ready for training!")
print("=" * 70)

## Data Preparation

### Load Synthetic Anomalies for Training

We load synthetic anomalies from Phase 1 (`synthetic-anomaly-generation.ipynb`) for training.

**Why Synthetic Data?**
- Real anomalies are rare (<1% in production clusters)
- Synthetic data provides labeled training examples
- Models learn general patterns, not memorize specific examples
- Balanced dataset (50% normal, 50% anomaly) improves performance
- Reproducible and testable

**Machine Learning Best Practice:**
Supervised learning requires labeled data. Synthetic data provides:
1. **Ground Truth**: Known labels for evaluation
2. **Balanced Classes**: Equal normal and anomaly samples
3. **Reproducibility**: Same data for consistent results
4. **Generalization**: Models learn patterns, not memorize examples

**References:**
- He & Garcia (2009): "Learning from Imbalanced Data" - https://ieeexplore.ieee.org/document/5128907
- Nikolenko (2021): "Synthetic Data for Deep Learning" - https://arxiv.org/abs/1909.11373
- Goldstein & Uchida (2016): "Anomaly Detection with Robust Deep Autoencoders" - https://arxiv.org/abs/1511.08747

In [ ]:
import requests
import os
from datetime import datetime, timedelta

# =============================================================================
# PROMETHEUS CLIENT (for loading real data)
# =============================================================================

class PrometheusDataLoader:
    """Load real metrics from Prometheus."""
    
    def __init__(self):
        token_path = '/var/run/secrets/kubernetes.io/serviceaccount/token'
        self.token = None
        if os.path.exists(token_path):
            with open(token_path, 'r') as f:
                self.token = f.read().strip()
        
        self.base_url = 'https://prometheus-k8s.openshift-monitoring.svc.cluster.local:9091'
        self.session = requests.Session()
        if self.token:
            self.session.headers.update({'Authorization': f'Bearer {self.token}'})
        self.session.verify = False
        
        import urllib3
        urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
        
        # Test connection
        try:
            response = self.session.get(f"{self.base_url}/api/v1/status/config", timeout=5)
            self.connected = response.status_code == 200
        except:
            self.connected = False
    
    def query_range(self, query, start, end, step='1m'):
        if not self.connected:
            return None
        
        url = f"{self.base_url}/api/v1/query_range"
        params = {'query': query, 'start': start, 'end': end, 'step': step}
        
        try:
            response = self.session.get(url, params=params, timeout=60)
            response.raise_for_status()
            return response.json()
        except:
            return None

# =============================================================================
# DATA PREPARATION FUNCTION
# =============================================================================

def prepare_anomaly_detection_data(duration_hours=24, use_real_data=True):
    """
    Prepare data for anomaly detection training.
    
    Args:
        duration_hours: Hours of data to collect
        use_real_data: If True, try to load from Prometheus. If False, use synthetic.
    
    Returns:
        dict: {metric_name: DataFrame with 'timestamp', 'value', 'is_anomaly'}
    """
    print("=" * 70)
    print("🔄 PREPARING ANOMALY DETECTION DATASET")
    print("=" * 70)
    print(f"   Duration: {duration_hours} hours")
    print(f"   Metrics: {len(TARGET_METRICS)}")
    print(f"   Use real data: {use_real_data}")
    
    all_data = {}
    data_sources = {}
    
    # Try to connect to Prometheus
    prometheus = None
    if use_real_data:
        prometheus = PrometheusDataLoader()
        print(f"\n📡 Prometheus connected: {prometheus.connected}")
    
    end_time = datetime.now()
    start_time = end_time - timedelta(hours=duration_hours)
    
    print(f"\n📊 Collecting metrics...")
    print("-" * 50)
    
    for i, metric in enumerate(TARGET_METRICS):
        print(f"   [{i+1}/{len(TARGET_METRICS)}] {metric}...", end=" ")
        
        # Try real data first
        real_data_loaded = False
        
        if prometheus and prometheus.connected and metric in PROMETHEUS_QUERIES:
            query = PROMETHEUS_QUERIES[metric]
            result = prometheus.query_range(
                query,
                start_time.timestamp(),
                end_time.timestamp(),
                step='1m'
            )
            
            if result and result.get('status') == 'success':
                data = result.get('data', {}).get('result', [])
                if data:
                    # Parse Prometheus response
                    rows = []
                    for series in data:
                        for ts, value in series.get('values', []):
                            rows.append({
                                'timestamp': pd.to_datetime(ts, unit='s'),
                                'value': float(value) if value != 'NaN' else np.nan,
                                'metric': metric,
                                'is_anomaly': False  # Will be labeled by model
                            })
                    
                    if rows:
                        df = pd.DataFrame(rows)
                        # Aggregate by timestamp (in case of multiple series)
                        df = df.groupby('timestamp').agg({
                            'value': 'mean',
                            'metric': 'first',
                            'is_anomaly': 'first'
                        }).reset_index()
                        
                        all_data[metric] = df
                        data_sources[metric] = 'REAL'
                        real_data_loaded = True
                        print(f"✅ REAL ({len(df)} points)")
        
        # Fallback to synthetic data
        if not real_data_loaded:
            df = generate_synthetic_timeseries(
                metric_name=metric,
                duration_hours=duration_hours,
                interval_minutes=1,
                add_anomalies=True,
                anomaly_probability=0.03
            )
            all_data[metric] = df
            data_sources[metric] = 'SYNTHETIC'
            print(f"📊 SYNTHETIC ({len(df)} points)")
    
    # Summary
    print("\n" + "=" * 70)
    print("📊 DATA COLLECTION SUMMARY")
    print("=" * 70)
    
    real_count = sum(1 for s in data_sources.values() if s == 'REAL')
    synthetic_count = sum(1 for s in data_sources.values() if s == 'SYNTHETIC')
    
    print(f"\n   Total metrics: {len(all_data)}")
    print(f"   ✅ REAL data: {real_count} metrics")
    print(f"   📊 SYNTHETIC data: {synthetic_count} metrics")
    
    total_points = sum(len(df) for df in all_data.values())
    total_anomalies = sum(df['is_anomaly'].sum() for df in all_data.values())
    
    print(f"\n   Total data points: {total_points:,}")
    print(f"   Total anomalies: {total_anomalies:,} ({total_anomalies/total_points:.2%})")
    
    if real_count > 0:
        print(f"\n   🎉 Using {real_count} REAL metrics from your OpenShift cluster!")
    
    return all_data, data_sources

# =============================================================================
# RUN DATA PREPARATION
# =============================================================================

# Set to True to try loading real Prometheus data
# Set to False to use only synthetic data (faster, for testing)
USE_REAL_PROMETHEUS_DATA = True

training_data, data_sources = prepare_anomaly_detection_data(
    duration_hours=24,  # Last 24 hours
    use_real_data=USE_REAL_PROMETHEUS_DATA
)

# Display what we got
print("\n📋 METRICS BY DATA SOURCE:")
print("-" * 50)
print("\n✅ REAL DATA:")
for metric, source in data_sources.items():
    if source == 'REAL':
        print(f"   {metric}: {len(training_data[metric])} points")

print("\n📊 SYNTHETIC DATA:")
for metric, source in data_sources.items():
    if source == 'SYNTHETIC':
        print(f"   {metric}: {len(training_data[metric])} points")


In [ ]:
def create_feature_matrix(data_dict):
    """
    Create feature matrix for anomaly detection
    """
    print("🔧 Creating feature matrix...")
    
    # Align all time series to common timestamps
    # Find common time range
    min_start = max(df['timestamp'].min() for df in data_dict.values())
    max_end = min(df['timestamp'].max() for df in data_dict.values())
    
    print(f"  📅 Time range: {min_start} to {max_end}")
    
    # Create common time index
    time_index = pd.date_range(start=min_start, end=max_end, freq='1min')
    
    # Build feature matrix
    features = pd.DataFrame(index=time_index)
    labels = pd.Series(index=time_index, dtype=bool, name='is_anomaly')
    
    for metric_name, df in data_dict.items():
        # Resample to common time index
        df_resampled = df.set_index('timestamp').reindex(time_index, method='nearest')
        
        # Add basic features
        features[f'{metric_name}_value'] = df_resampled['value']
        
        # Add rolling statistics (5-minute windows)
        features[f'{metric_name}_mean_5m'] = df_resampled['value'].rolling('5min').mean()
        features[f'{metric_name}_std_5m'] = df_resampled['value'].rolling('5min').std()
        features[f'{metric_name}_min_5m'] = df_resampled['value'].rolling('5min').min()
        features[f'{metric_name}_max_5m'] = df_resampled['value'].rolling('5min').max()
        
        # Add lag features
        features[f'{metric_name}_lag_1'] = df_resampled['value'].shift(1)
        features[f'{metric_name}_lag_5'] = df_resampled['value'].shift(5)
        
        # Add rate of change
        features[f'{metric_name}_diff'] = df_resampled['value'].diff()
        features[f'{metric_name}_pct_change'] = df_resampled['value'].pct_change()
        
        # Combine anomaly labels (any metric anomaly = overall anomaly)
        metric_anomalies = df_resampled['is_anomaly'].fillna(False)
        labels = labels | metric_anomalies
    
    # Fill missing values
    features = features.ffill().bfill()
    labels = labels.fillna(False)
    
    # Replace infinity values with 0 and remaining NaN with 0
    features = features.replace([np.inf, -np.inf], 0)
    features = features.fillna(0)
    
    print(f"  ✅ Feature matrix: {features.shape}")
    print(f"  🏷️ Anomaly labels: {labels.sum()} anomalies ({labels.mean():.2%})")
    
    return features, labels

# Create feature matrix
X, y = create_feature_matrix(training_data)

print(f"\n📊 Feature Engineering Complete:")
print(f"  Features: {X.shape[1]} columns")
print(f"  Samples: {X.shape[0]:,} rows")
print(f"  Anomaly rate: {y.mean():.2%}")

## Model Training and Evaluation

Train Isolation Forest model and evaluate its performance.

In [ ]:
# Split data for training and testing
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.3, random_state=42, stratify=y
)

print(f"📊 Data Split:")
print(f"  Training: {X_train.shape[0]:,} samples")
print(f"  Testing: {X_test.shape[0]:,} samples")
print(f"  Training anomalies: {y_train.sum()} ({y_train.mean():.2%})")
print(f"  Testing anomalies: {y_test.sum()} ({y_test.mean():.2%})")

# ✨ Create sklearn Pipeline combining scaler + model (KServe compatible)
print("\n🔧 Creating Isolation Forest pipeline (scaler + model)...")
print("   This creates a SINGLE .pkl file for KServe deployment")

isolation_forest_pipeline = Pipeline([
    ('scaler', RobustScaler()),  # More robust to outliers than StandardScaler
    ('isolation_forest', IsolationForest(**ISOLATION_FOREST_CONFIG))
])

print("✅ Pipeline created (scaler + Isolation Forest)")

In [ ]:
# Train Isolation Forest Pipeline
print("🌲 Training Isolation Forest pipeline...")
print("   Pipeline automatically handles: scaler.fit_transform() → model.fit()")

# Fit pipeline on training data
# Pipeline will:
#   1. Fit scaler on X_train and transform it
#   2. Fit Isolation Forest on scaled data
isolation_forest_pipeline.fit(X_train)

print("✅ Training complete")

# Make predictions using pipeline
# Pipeline will:
#   1. Transform data using fitted scaler
#   2. Predict using fitted model
print("🔮 Making predictions...")
y_pred_train = isolation_forest_pipeline.predict(X_train)
y_pred_test = isolation_forest_pipeline.predict(X_test)

# Get anomaly scores
train_scores = isolation_forest_pipeline.decision_function(X_train)
test_scores = isolation_forest_pipeline.decision_function(X_test)

# Convert predictions to binary (1 = normal, -1 = anomaly)
y_pred_train_binary = (y_pred_train == -1)
y_pred_test_binary = (y_pred_test == -1)

print(f"  Training predictions: {y_pred_train_binary.sum()} anomalies detected")
print(f"  Testing predictions: {y_pred_test_binary.sum()} anomalies detected")
print("\n✅ Pipeline handles scaling automatically - no separate scaler needed!")

In [ ]:
# Evaluate model performance
print("📊 Model Evaluation")
print("=" * 50)

# Training set performance
print("\n🏋️ Training Set Performance:")
print(classification_report(y_train, y_pred_train_binary, 
                          target_names=['Normal', 'Anomaly']))

# Test set performance
print("\n🧪 Test Set Performance:")
print(classification_report(y_test, y_pred_test_binary, 
                          target_names=['Normal', 'Anomaly']))

# Confusion matrices
fig, axes = plt.subplots(1, 2, figsize=(12, 5))

# Training confusion matrix
cm_train = confusion_matrix(y_train, y_pred_train_binary)
sns.heatmap(cm_train, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomaly'], 
            yticklabels=['Normal', 'Anomaly'], ax=axes[0])
axes[0].set_title('Training Set Confusion Matrix')
axes[0].set_ylabel('True Label')
axes[0].set_xlabel('Predicted Label')

# Test confusion matrix
cm_test = confusion_matrix(y_test, y_pred_test_binary)
sns.heatmap(cm_test, annot=True, fmt='d', cmap='Blues', 
            xticklabels=['Normal', 'Anomaly'], 
            yticklabels=['Normal', 'Anomaly'], ax=axes[1])
axes[1].set_title('Test Set Confusion Matrix')
axes[1].set_ylabel('True Label')
axes[1].set_xlabel('Predicted Label')

plt.tight_layout()
plt.show()

## Model Analysis and Visualization

In [ ]:
# Analyze anomaly scores distribution
fig, axes = plt.subplots(2, 2, figsize=(15, 10))
fig.suptitle('Isolation Forest Analysis', fontsize=16, fontweight='bold')

# Score distribution
axes[0, 0].hist(train_scores[~y_train], bins=50, alpha=0.7, label='Normal', density=True)
axes[0, 0].hist(train_scores[y_train], bins=50, alpha=0.7, label='Anomaly', density=True)
axes[0, 0].set_title('Anomaly Score Distribution (Training)')
axes[0, 0].set_xlabel('Anomaly Score')
axes[0, 0].set_ylabel('Density')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Score vs time (sample)
sample_size = min(1000, len(test_scores))
sample_indices = np.random.choice(len(test_scores), sample_size, replace=False)
sample_indices = np.sort(sample_indices)

axes[0, 1].plot(sample_indices, test_scores[sample_indices], 'b-', alpha=0.7, linewidth=1)
anomaly_indices = sample_indices[y_test.iloc[sample_indices]]
if len(anomaly_indices) > 0:
    axes[0, 1].scatter(anomaly_indices, test_scores[anomaly_indices], 
                      color='red', s=30, alpha=0.8, label='True Anomalies')
axes[0, 1].axhline(y=0, color='gray', linestyle='--', alpha=0.5)
axes[0, 1].set_title('Anomaly Scores Over Time (Test Sample)')
axes[0, 1].set_xlabel('Sample Index')
axes[0, 1].set_ylabel('Anomaly Score')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Feature importance (using PCA to visualize)
pca = PCA(n_components=2)
X_test_pca = pca.fit_transform(X_test)

normal_mask = ~y_test
anomaly_mask = y_test

axes[1, 0].scatter(X_test_pca[normal_mask, 0], X_test_pca[normal_mask, 1], 
                  c='blue', alpha=0.6, s=20, label='Normal')
axes[1, 0].scatter(X_test_pca[anomaly_mask, 0], X_test_pca[anomaly_mask, 1], 
                  c='red', alpha=0.8, s=30, label='Anomaly')
axes[1, 0].set_title('PCA Visualization (Test Set)')
axes[1, 0].set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.1%} variance)')
axes[1, 0].set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.1%} variance)')
axes[1, 0].legend()
axes[1, 0].grid(True, alpha=0.3)

# Model performance metrics
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score

precision = precision_score(y_test, y_pred_test_binary)
recall = recall_score(y_test, y_pred_test_binary)
f1 = f1_score(y_test, y_pred_test_binary)

# Convert scores to probabilities for AUC calculation
test_scores_prob = (test_scores - test_scores.min()) / (test_scores.max() - test_scores.min())
auc = roc_auc_score(y_test, 1 - test_scores_prob)  # Invert because lower scores = more anomalous

metrics_text = f"""
Model Performance Metrics:

Precision: {precision:.3f}
Recall: {recall:.3f}
F1-Score: {f1:.3f}
AUC-ROC: {auc:.3f}

Configuration:
Trees: {ISOLATION_FOREST_CONFIG['n_estimators']}
Contamination: {ISOLATION_FOREST_CONFIG['contamination']}
Features: {X.shape[1]}

Data:
Training: {X_train.shape[0]:,}
Testing: {X_test.shape[0]:,}
"""

axes[1, 1].text(0.05, 0.95, metrics_text, transform=axes[1, 1].transAxes, 
               fontsize=10, verticalalignment='top',
               bbox=dict(boxstyle='round', facecolor='lightgray', alpha=0.8))
axes[1, 1].set_title('Model Summary')
axes[1, 1].axis('off')

plt.tight_layout()
plt.show()

print(f"\n🎯 Model Performance Summary:")
print(f"  Precision: {precision:.3f}")
print(f"  Recall: {recall:.3f}")
print(f"  F1-Score: {f1:.3f}")
print(f"  AUC-ROC: {auc:.3f}")

## Save Model and Upload to S3

In [ ]:
# Save pipeline model to persistent storage
# Use /mnt/models for persistent storage (model-storage-pvc)
# Fallback to local for development outside cluster
MODELS_DIR = Path('/mnt/models') if Path('/mnt/models').exists() else Path('/opt/app-root/src/models')

# Create KServe-compatible subdirectory structure
MODEL_NAME = 'anomaly-detector'
MODEL_DIR = MODELS_DIR / MODEL_NAME
MODEL_DIR.mkdir(parents=True, exist_ok=True)

# Save with KServe expected filename
model_path = MODEL_DIR / 'model.pkl'

# Migration: Move old flat file if exists
old_path = MODELS_DIR / 'anomaly-detector.pkl'
if old_path.exists() and not model_path.exists():
    import shutil
    shutil.move(str(old_path), str(model_path))
    print(f"🔄 Migrated model from {old_path} to {model_path}")

# ✨ Save SINGLE pipeline file (KServe compatible)
# KServe sklearn server expects model at: /mnt/models/anomaly-detector/model.pkl
joblib.dump(isolation_forest_pipeline, model_path)
print(f"💾 Saved Isolation Forest pipeline to: {model_path}")
print(f"   ✅ KServe-compatible path: {MODEL_NAME}/model.pkl")
print(f"   ✅ Single .pkl file (scaler + model combined)")

# Upload model to S3 for persistent storage
try:
    from common_functions import upload_model_to_s3, test_s3_connection
    
    if test_s3_connection():
        upload_model_to_s3(
            str(model_path),
            s3_key='models/anomaly-detection/anomaly-detector/model.pkl'
        )
        print(f"☁️  Uploaded to S3: models/anomaly-detection/anomaly-detector/model.pkl")
    else:
        print("⚠️ S3 not available - model saved locally only")
except ImportError:
    print("⚠️ S3 functions not available - model saved locally only")
except Exception as e:
    print(f"⚠️ S3 upload failed (non-critical): {e}")

# Verify model saved
assert model_path.exists(), "Pipeline model not saved"
print("\n✅ Model pipeline saved successfully")
print(f"   Path: {model_path}")
print(f"   Size: {model_path.stat().st_size / 1024:.2f} KB")

# Clean up old separate model/scaler files if they exist
old_model = MODELS_DIR / 'isolation_forest_model.pkl'
old_scaler = MODELS_DIR / 'isolation_forest_scaler.pkl'
for old_file in [old_model, old_scaler]:
    if old_file.exists():
        old_file.unlink()
        print(f"🗑️  Removed old file: {old_file.name}")